In [37]:
import requests
from bs4 import BeautifulSoup
import functools
from function_pipes import pipe
import numpy as np
import networkx as nx
import copy
import matplotlib
from pprint import pprint
import pandas as pd
import collections

Grabbing Webpage

In [38]:
url = "https://masteringruneterra.com/mu-table/"
page = requests.get(url)

soup = BeautifulSoup(page.content, 'html.parser')

Doing finding and stuff

In [39]:
headers = soup.select("div.grid-item:not(.versus-cell)")
headers = map(lambda x: x.text.strip(), headers)
decks = list(headers)[1:16]
print(len(decks))
print(decks)

15
['Annie Jhin (NX/RU)', 'Gwen Zed (IO/SI)', 'Elder Dragon Mordekaiser Viego (RU/SI)', "Pyke Rek'Sai (BW/SH)", 'Elder Dragon Galio Morgana (DE/RU)', 'Caitlyn Teemo (FR/PZ)', 'Ashe LeBlanc (FR/NX)', 'Master Yi Nami (BW/IO)', 'Elder Dragon Volibear (FR/RU)', 'Aurelion Sol Elder Dragon (MT/RU)', 'Karma Sett (IO/PZ)', 'Elder Dragon Morgana Shyvana (DE/RU)', 'Karma Sett (FR/IO)', 'Janna Nilah (BW/PZ)', 'Maokai Nautilus (BW/SI)']


In [57]:
def handleWinrate(winRate: str) -> str:
    winRate = winRate[:-1]  #dropping %
    winRate = float(winRate)
    winRate = winRate / 100
    winRate = round(winRate, 4)
    return winRate


In [40]:

cells = soup.select(".versus-cell > div")
rawText = map(lambda x: x.text.strip(), cells)


def getWinRateAndGames(text):
    winRate, games = text.split("\n");
    #handing winrate
    winRate = handleWinrate(winRate)

    #handling games
    games = games.split(" ")[1]
    games = int(games)
    return {"winRate": winRate, "gamesPlayed": games}


matchUps = list(map(getWinRateAndGames, rawText))
matchUps[0:10]



[{'winRate': 0.616, 'gamesPlayed': 526},
 {'winRate': 0.6267, 'gamesPlayed': 442},
 {'winRate': 0.556, 'gamesPlayed': 464},
 {'winRate': 0.5016, 'gamesPlayed': 315},
 {'winRate': 0.7421, 'gamesPlayed': 252},
 {'winRate': 0.6157, 'gamesPlayed': 281},
 {'winRate': 0.6199, 'gamesPlayed': 221},
 {'winRate': 0.7647, 'gamesPlayed': 238},
 {'winRate': 0.6434, 'gamesPlayed': 244},
 {'winRate': 0.4661, 'gamesPlayed': 221}]

Converting to a table

In [79]:
table = {key: {} for key in decks}
muIndex = 0
for rawIndex in range(len(decks) * len(decks)):
    i = rawIndex // 15
    j = rawIndex % 15
    deck1 = decks[i]
    deck2 = decks[j]
    if i == j:  #diagonal on the match up, the deck is playing itself
        table[deck1][deck2] = {"win_rate": 0.5, "games_played": 10000}
        continue
    mu = matchUps[muIndex]
    muIndex += 1
    table[deck1][deck2] = {"win_rate": mu["winRate"], "games_played": mu["gamesPlayed"]}
print(table)

{'Annie Jhin (NX/RU)': {'Annie Jhin (NX/RU)': {'win_rate': 0.5, 'games_played': 10000}, 'Gwen Zed (IO/SI)': {'win_rate': 0.616, 'games_played': 526}, 'Elder Dragon Mordekaiser Viego (RU/SI)': {'win_rate': 0.6267, 'games_played': 442}, "Pyke Rek'Sai (BW/SH)": {'win_rate': 0.556, 'games_played': 464}, 'Elder Dragon Galio Morgana (DE/RU)': {'win_rate': 0.5016, 'games_played': 315}, 'Caitlyn Teemo (FR/PZ)': {'win_rate': 0.7421, 'games_played': 252}, 'Ashe LeBlanc (FR/NX)': {'win_rate': 0.6157, 'games_played': 281}, 'Master Yi Nami (BW/IO)': {'win_rate': 0.6199, 'games_played': 221}, 'Elder Dragon Volibear (FR/RU)': {'win_rate': 0.7647, 'games_played': 238}, 'Aurelion Sol Elder Dragon (MT/RU)': {'win_rate': 0.6434, 'games_played': 244}, 'Karma Sett (IO/PZ)': {'win_rate': 0.4661, 'games_played': 221}, 'Elder Dragon Morgana Shyvana (DE/RU)': {'win_rate': 0.4103, 'games_played': 195}, 'Karma Sett (FR/IO)': {'win_rate': 0.7401, 'games_played': 177}, 'Janna Nilah (BW/PZ)': {'win_rate': 0.298, 'g

Scraping overall play and winrates

In [46]:
deckCells = soup.select(".grid-container:not(:first-child):not(:last-child) > .grid-item.side-header:first-child")
len(deckCells)

15

In [74]:
from dataclasses import dataclass, asdict


@dataclass
class Archetype:
    name: str
    games_played: int
    win_rate: float
    play_rate: float


def parseDeckCell(deckCell):
    children = list(deckCell.childGenerator())
    children = [child.text for child in children]
    name = children[0]

    games_played = children[1].split(" ")[1]
    games_played = int(games_played)

    win_rate = children[2].split(" ")[1]
    win_rate = handleWinrate(win_rate)

    play_rate = children[4].split(" ")[1]
    play_rate = handleWinrate(play_rate)

    ans = Archetype(name, games_played, win_rate, play_rate)
    return ans


archeTypes = [parseDeckCell(cell) for cell in deckCells]

# Sending to DB

In [85]:
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os

load_dotenv()

db_url = os.environ.get("DB_URI")
engine = create_engine(db_url, pool_pre_ping=True);

Inserting archetypes

In [88]:
def insert_decks_and_matchups(archeTypes, table):
    with engine.begin() as connection:
        #clearing table first
        cmd = "TRUNCATE archetypes, matchups"
        connection.execute(text(cmd))
        
        #inserting decks
        for archeType in archeTypes:
            cmd = "insert into archetypes (NAME, games_played, win_rate, play_rate) VALUES (:name, :games_played, :win_rate, :play_rate)"
            connection.execute(text(cmd), asdict(archeType))
            
        #inserting matchups
        for playing_deck, opposing_decks in table.items():
            for opposing_deck, data in opposing_decks.items():
                cmd = """INSERT INTO matchups (playing, opposing, win_rate, games_played) VALUES (
                    (SELECT id from archetypes WHERE
                        archetypes.name = :playing_name),
                    (SELECT id from archetypes WHERE
                        archetypes.name = :opposing_name),
                    :win_rate,
                    :games_played
                    )
                """
                binds = {"playing_name": playing_deck, "opposing_name": opposing_deck, "win_rate": data["win_rate"],
                         "games_played": data["games_played"]}
                connection.execute(text(cmd), binds)


In [89]:
insert_decks_and_matchups(archeTypes, table)